# Exploratory Data Analysis

## Import Libraries

In [8]:
########################### Import Requisite Libraries #########################
import pandas as pd
import numpy as np
import os
import sys
import pprint

################################################################################
# Add the parent directory to sys.path to access 'functions.py'
sys.path.append(os.path.join(os.pardir))
from python_scripts.functions import *

## Read in the Data

In [9]:
base_path = os.path.join(os.pardir)

# Go up one level from 'notebooks' to the parent directory, then into the 'data' folder
data_path = os.path.join(os.pardir, "data")

image_path_png = os.path.join(base_path, "images", "png_images")
image_path_svg = os.path.join(base_path, "images", "svg_images")

# Use the function to ensure the 'data' directory exists
ensure_directory(data_path)
ensure_directory(image_path_png)
ensure_directory(image_path_svg)

# Read the CSV file into a DataFrame
df_eda = pd.read_parquet(os.path.join(data_path, "df_eda.parquet"))

Directory exists: ..\data
Directory exists: ..\images\png_images
Directory exists: ..\images\svg_images


In [11]:
df_eda.head()

,Attendance date,Age,SEX,ETHNICITY,Diabetes (1=yes; 0=no),Hypertension (1=yes; 0=no),"Renal disease (DM=1, HTN=2, GN=3, ADPKD=4, Other=5)",eGFR-EPI,uPCR,Calcium (mmol/L),...,ETHNICITY_BANGLADESHI (ASIAN OR ASIAN BRITISH),ETHNICITY_CARIBBEAN (BLACK OR BLACK BRITISH),ETHNICITY_CHINESE (OTHER ETHNIC GROUPS),ETHNICITY_INDIAN (ASIAN OR ASIAN BRITISH),ETHNICITY_NOT STATED,ETHNICITY_PAKISTANI (ASIAN OR ASIAN BRITISH),ETHNICITY_WHITE AND BLACK AFRICAN (MIXED),ETHNICITY_WHITE BRITISH,ETHNICITY_WHITE IRISH,age_group
Patient_ID,,,,,,,,,,,,,,,,,,,,,
568268916,11/02/2015,87.24,Male,WHITE BRITISH,1.0,1.0,1.0,19.0,33.0,2.78,...,0,0,0,0,0,0,0,1,0,80-89
659549633,06/05/2015,56.88,Female,WHITE BRITISH,0.0,1.0,5.0,15.0,395.0,2.43,...,0,0,0,0,0,0,0,1,0,50-59
406748956,04/05/2016,66.53,Female,WHITE BRITISH,0.0,1.0,3.0,17.0,163.0,2.33,...,0,0,0,0,0,0,0,1,0,60-69
109228713,16/11/2011,69.92,Male,WHITE IRISH,0.0,1.0,3.0,12.0,250.0,2.29,...,0,0,0,0,0,0,0,0,1,60-69
220533110,02/05/2018,81.14,Female,WHITE BRITISH,1.0,1.0,1.0,15.0,217.0,2.45,...,0,0,0,0,0,0,0,1,0,80-89
